# 第9回: RNN: Exercise2
## シェイクスピアをCharacter-Level RNNで学習する
### 概要
本演習ではChainerを用いてCharacter Level RNNを実装します。

まず、講師が概要を説明しますので、全体を掴んだところで演習に取り組んでください。
### 目標
- ChainerでRNNを実装する
- 言語モデルを学習させる
- truncated backpropを実装する

### おまじない
必要なモジュールをimportしましょう。

In [1]:
%matplotlib inline
import chainer
from chainer import serializers
from chainer import cuda, Function, optimizers
from chainer import Link, Chain, ChainList, Variable
import chainer.functions as F
import chainer.links as L
import matplotlib.pyplot as plt
import tqdm
from tqdm import tqdm_notebook

<div class="alert alert-block alert-info">
Note:<br>
以下でCPU or GPUを使うか選択してください。
</div>

In [2]:
import numpy as np
xp = np  # for CPU computations

"""
import cupy as cp
xp = cp  # for GPU computations
"""

'\nimport cupy as cp\nxp = cp  # for GPU computations\n'

## Character Level言語モデル

ある入力列$x_1, x_2, \dots, x_t$が与えられたときに、
$x_{t+1}$を予測するモデルを文字レベルの言語モデル(character level language model)と呼びます。

たとえば、'hello'を学習したCharacter Level言語モデルは、'hell'が与えられたときに、次に現れるアルファベット'o'を予測する事ができます。

本演習では、Shakespeareの悲劇の一つである『リア王』を用いて、Character Level Language Modelを学習させてみます。

## 学習データの構造と前処理
### 学習データの構造
さて、学習に用いるデータセットは、例えば以下のような文章で構成されています。

```
BIONDELLO:
Marry, that it may not pray their patience.'

KING LEAR:
The instant common maid, as we may less be
a brave gentleman and joiner: he that finds us with wax
And owe so full of presence and our fooder at our
staves. It is remorsed the bridal's man his grace
for every business in my tongue, but I was thinking
that he contends, he hath respected thee.
```

文章を実際に扱うには、幾つかのプログラム上の工夫が必要になります。
まず、学習データ中で使用される文字や記号をリカレントニューラルネットワークに入力する際に、これらの文字や記号をベクトルで表記する必要があります。例えば、'b'という文字をネットワークに入力する際には、アルファベットの二番目の文字なので、`...00010`というように、下から2bit目を立たせて表現します。

このように、記号をネットワークで扱いやすいような
分散表現に変換する操作のことをword embeddings(単語の分散表現)と呼びます。
実際に言語モデルや翻訳モデルを獲得する際には、
このように文字(character)を変換するのではなく、よりまとまった表現として語(word)を分散表現に変換する
Word to Vector(W2V)という手法も広く使われることも覚えておくと良いでしょう。

Chainerはこうした文字埋め込みの分散表現が`links.EmbedID`に実装されています。
例えば、`abc...xyz`(小文字アルファベット、
23文字)から構成される、任意の時系列データを学習させる場合の前処理は、
以下のように記述できます。

##### Example
```python
embed = L.EmbedID(23, l1_hidden_units)
```

EmbedIDでは入力をone-hotベクトルとして、次の層に渡す役割を担います。

### 前処理(preprocessing)

今回使用するデータセットは、アルファベット(大文字、小文字)、
及び句読記号(コンマ、コロン、セミコロンなど)にから構成されそうです。

実際にデータ・セットを読み込んで確かめてみましょう。

In [3]:
# load text file
with open('./data/RNN/shakespear.txt', 'r') as full_text:
    full_text = open('./data/RNN/shakespear.txt', 'r').readlines()

In [8]:
# check how it looks like (if you want)
# print(full_text)

さて、`list`に格納された`full_text`を学習データセットとして扱うには少し工夫が必要です。

すなわち、
- 文字をone-hot表現に変換
- 段落の終わりを告げる`<EOP>`を導入
- 行の終わりを告げる`<EOL>`を導入

する必要があります。

まず、文字のone-hot表現への変換ですが、これは文字と(uniqueな)IDの一対一対応を持つ辞書を作成すれば良いでしょう。

また、`<EOP>`, `<EOS>`の導入も上記の辞書を使用することで解決できますが、
これによって、データをネットワークにfeedする際に区切りをつけて処理を行ったり、改行を扱ったりすることが出来ます。

この前処理(preprocessing)は、以下のように実装されます。

In [4]:
symbol = {'<EOL>': 0, '<EOP>': 1}
dataset = []
for line in full_text:
    if line == '\n':
        dataset.append(Variable(xp.array([symbol['<EOP>']], dtype=xp.int32)))
        continue
    for letter in line.replace('\n', ''):
        if not letter in symbol:
            symbol[letter] = len(symbol)
        dataset.append(Variable(xp.array([symbol[letter]], dtype=xp.int32)))
    dataset.append(Variable(xp.array([symbol['<EOL>']], dtype=xp.int32)))

In [5]:
# symbol dict: symbols -> id
print(symbol)

{'<EOL>': 0, '<EOP>': 1, 'T': 2, 'h': 3, 'a': 4, 't': 5, ',': 6, ' ': 7, 'p': 8, 'o': 9, 'r': 10, 'c': 11, 'n': 12, 'e': 13, 'm': 14, 'l': 15, 'i': 16, "'": 17, 'd': 18, 'u': 19, 's': 20, 'f': 21, 'I': 22, 'y': 23, 'v': 24, ';': 25, 'q': 26, 'H': 27, 'b': 28, 'k': 29, 'x': 30, 'A': 31, 'B': 32, 'w': 33, 'g': 34, 'G': 35, '.': 36, 'O': 37, 'N': 38, 'D': 39, 'E': 40, 'L': 41, ':': 42, 'M': 43, 'K': 44, 'R': 45, 'j': 46, 'S': 47, '-': 48, 'F': 49, 'W': 50, 'U': 51, 'Y': 52, 'C': 53, 'V': 54, 'P': 55, '?': 56, '!': 57, 'J': 58, 'X': 59, 'z': 60, 'Q': 61, 'Z': 62}


In [6]:
# inv dict: id -> symbols
symbol_inv = {v: k for k, v in symbol.items()}
print(symbol_inv)

{0: '<EOL>', 1: '<EOP>', 2: 'T', 3: 'h', 4: 'a', 5: 't', 6: ',', 7: ' ', 8: 'p', 9: 'o', 10: 'r', 11: 'c', 12: 'n', 13: 'e', 14: 'm', 15: 'l', 16: 'i', 17: "'", 18: 'd', 19: 'u', 20: 's', 21: 'f', 22: 'I', 23: 'y', 24: 'v', 25: ';', 26: 'q', 27: 'H', 28: 'b', 29: 'k', 30: 'x', 31: 'A', 32: 'B', 33: 'w', 34: 'g', 35: 'G', 36: '.', 37: 'O', 38: 'N', 39: 'D', 40: 'E', 41: 'L', 42: ':', 43: 'M', 44: 'K', 45: 'R', 46: 'j', 47: 'S', 48: '-', 49: 'F', 50: 'W', 51: 'U', 52: 'Y', 53: 'C', 54: 'V', 55: 'P', 56: '?', 57: '!', 58: 'J', 59: 'X', 60: 'z', 61: 'Q', 62: 'Z'}


ネットワークに与えられる入力列(dataset)は以下のように、記号に対応したidに置き換えられた形になります

```python
[2, 3, 4, 5, 6, 7, 8, 9, 9, 10, 7, 11, 9, ...
```

この入力列は実際には、
```python
['T', 'h', 'a', 't', ',', ' ', 'p', 'o', 'o', 'r', ' '
 ```
を表しています。(気になる方は以下のサンプルコードを実行してみてください)

In [14]:
# below won't work for gpu
# print([v.data[0] for v in dataset])

In [19]:
# below won't work for gpu
# print([symbol_inv[v.data[0]] for v in dataset][0:400])

<div class="alert alert-block alert-info">
Note:<br>
以上がネットワークにどのような情報を与えるのか決める、
前処理(preprocessing)の作業となります。
言語処理においてどのように前処理を行うのかという問題は、
望ましい出力を得るために重要となります。
</div>

## Character Level言語モデルを学習するRNNの実装
では、実際にCharacter Level Language Modelを学習するRNNを
`Chainer`で実装していきましょう。

今回は、学習データの関係上、二層のRNNを使用して実装を行います。

In [7]:
class CLRNN(Chain):
    def __init__(
        self,
        n_input,
        n_hidden,
        n_output,
    ):
        """
        Initialize CLRNN
        n_input: # of characters
        n_hidden: # of hidden units
        n_output: # of outputs, equals to # of input in this case
        """
        super(CLRNN, self).__init__()
        with self.init_scope():
            '''
                YOUR CODE HERE
            '''
            
    def __call__(self, x):
        """
        Forward computation.
        x: input vector (in one hot representation)
        """
        x = "*** YOUR CODE HERE ***"
        h1 = "*** YOUR CODE HERE ***"
        h2 = "*** YOUR CODE HERE ***"
        y = "*** YOUR CODE HERE ***"
        return y
    
    def reset_state(self):
        '''
        a function to clear the states in the hidden layer
        '''
        self.lstm.reset_state()
        self.lstm2.reset_state()

## truncated BPの実装
### BPTTの問題
非常に長いシークエンスを学習することは、
勾配計算をかなり前まで遡らなければならないことを意味します。
こうした勾配計算は講義で解説したBPTT(Back Propagation Through Time)
として実行されますが、

* 計算過程で勾配消失・爆発を引き起こす
* (長期にわたる勾配計算では誤差計算を保持しなければならないため)メモリリソースを圧迫する

という点で問題があります。
これらの問題を解決するために、
しばしばバックプロパゲーションを短い時間範囲で切り捨てる場合があります。
この手法はtruncated backpropagationと呼ばれます。
これはヒューリスティックな手法であり、もちろん切り捨てられた勾配は失われますが、
上記のような問題を上手く解決し、結果的に学習効率を向上させることが出来ます。

### truncated BP

`Chainer`にはtruncated backpropagationの実装として、
`Variable.unchain_backward()`というメソッドが実装されています。

`Backward Unchaining`は変数から計算履歴(勾配計算)の履歴を断ち切ります。

`truncated backpropagation`の例を以下に示します。

In [8]:
rnn = CLRNN("*** YOUR CODE HERE ***")
model = L.Classifier(rnn)
optimizer = optimizers.SGD()
optimizer.setup(model)

In [135]:
# !!! FOR GPU !!!
gpu_device = 0
cuda.get_device(gpu_device).use()
model.to_gpu(gpu_device)

<div class="alert alert-block alert-info">
Note:<br>
GPUを使用して学習する場合は、上記のコードを実行してください。
</div>

In [9]:
# train rnn with truncated backpropagation
def partial_fit():
    loss = 0
    count = 0
    seqlen = len(dataset)

    rnn.reset_state()
    trange = tqdm_notebook(
        range(seqlen-1),
        leave=False
    )
    for i in trange:
        "*** YOUR CODE HERE ***"
        "*** YOUR CODE HERE ***"
        loss += model(cur_word, next_word)
        count += 1
        if count % 100 == 0 or count == seqlen:
            model.cleargrads()
            loss.backward()
            loss.unchain_backward()
            optimizer.update()
    print('Train Error: {:4f}'.format(float(loss.data)))
    return loss

各ステップでの誤差は`loss`に蓄積されています。
今回の実装では、100stepごとに蓄積された誤差に対してバックプロパゲーションを行うと同時に、
`loss.unchain_backward()`メソッドを呼び出すことで、
蓄積された損失を計算履歴から消去(切り捨て)しています。

## 学習
では実際に学習してみましょう。

CLRNNの学習には非常に時間がかかります。学習回数は5-10epoch程度が良いでしょう。

In [11]:
epoch_loss = []
for i in tqdm_notebook(range(3), leave=False):
    epoch_loss.append(partial_fit())

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


KeyboardInterrupt: 

785/|/  1%|| 785/99992 [00:13<29:09, 56.72it/s]

## 順伝搬の計算
### 計算履歴を保存しないネットワーク評価
RNNの順伝搬計算で必要なのは、入力及び一時刻前の隠れ層の状態のみで、
逆伝搬のように計算履歴をすべて保存する必要はありません。
`Chainer`では計算履歴を保存しないForward用の`chainer.config.enable_backprop`という`flag`を用意しています。(Chainer v1で使用されていた`volatile option`はv2で[廃止されました]((http://docs.chainer.org/en/stable/upgrade.html?highlight=volatile#volatile-flag-is-removed))

##### Example
```python
with chainer.no_backprop_mode():
    x = Variable(x_data)
    feat = fixed_func(x)
y = predictor_func(feat)
y.backward()

```

<div class="alert alert-block alert-warning">
Warning:<br>
no_backprop_modeを使用した場合、計算グラフに値が保存されなくなります。それに伴い、loss.backward()は呼び出せなくなります。
</div>

では、例に習って順伝搬の計算を実装してみましょう。

In [131]:
rnn.reset_state()
out = []
with chainer.no_backprop_mode():
    for i in range(10):
        rnn(dataset[i])
    for i in range(400):
        x = Variable(xp.array([rnn(x).data.argmax()], dtype=xp.int32))
        out.append(x)

In [132]:
predicted = [int(o.data) for o in out]
for cid in predicted:
    if cid == symbol['<EOL>'] or cid==symbol['<EOP>']:
        print('')
        continue
    print(symbol_inv[cid], end='')

e before the say to the say
That were shall be despised of thy soul be despiritse a deed, when an his story:
The shall be so be soul be despiritse a deed, where he say the fortune and say and say to the speak were and speak with the say
That were shall be despised of thy soul be despiritse a deed, when an his story:
The shall be so be soul be despiritse a deed, where he say the fortune and say and

## 学習データの読み込み
### 学習済みデータの読み込み
CLRNNの学習はGPUを使用していても時間がかかります。
すでに学習済みのモデルがありますので、そちらを読み込んで遊んでみましょう。

事前に学習したモデルは`.npz`という`pickle`形式で保存されています。モデルの読み込みには`chainer.serializers`を使用します。
学習した時と同じ設定でインスタンスを作らなければならない点に注意しましょう。

In [13]:
rnn_loaded = CLRNN(n_input=len(symbol), n_hidden=1000, n_output=len(symbol))
model = L.Classifier(rnn_loaded)
serializers.load_npz("./data/RNN/clrnn.npz", model)

In [15]:
rnn_loaded.reset_state()
out = []
"*** YOUR CODE HERE ***"

contempt, or claim'd thou slept so faithful,
I may contrive our father; and, in their defeated queen,
Her flesh broke me and puttance of expedition house,
And in that same that ever I lament this stomach,
And he, nor Butly and my fury, knowing everything
Grew daily ever, his great strength and thought
The bright buds of mine own.

BIONDELLO:
Marry, that it may not pray their patience.'

KING LEAR: